## Installing libraries

In [ ]:
!pip install cohere
!pip install langchain
!pip install pypdf
!pip install tiktoken
!pip install chromadb
!pip install bs4

## Use Cohere API endpoint

In [52]:
import cohere
key = "urAN0hpVmnaPAW66RPqZQTKOSs46zd4EI93e1NNg"
co = cohere.Client(key)
response = co.generate(
  model='command-nightly',
  prompt='write a description about hong kong',
  max_tokens=300,
  temperature=0.9,
  k=0,
  p=0.75,
  stop_sequences=[],
  return_likelihoods='NONE')
print('Prediction: {}'.format(response.generations[0].text))

Prediction: Hong Kong is a vibrant and bustling city located on the southeastern coast of China. Known for its stunning skyline, bustling streets, and diverse culture, Hong Kong is a unique and exciting destination that blends traditional Chinese heritage with modern influences.

The city is made up of Hong Kong Island, Kowloon, and the New Territories, each offering its own distinct character and attractions. Hong Kong Island is home to the city's financial district, with towering skyscrapers and bustling streets. Kowloon is a lively and densely populated area known for its vibrant markets and temples. The New Territories offer a more relaxed pace of life, with beautiful countryside, traditional villages, and stunning beaches.

One of the most iconic features of Hong Kong is its stunning skyline, which can be best appreciated from the famous Victoria Peak. From the peak, visitors can take in breathtaking views of the city's towering skyscrapers, sparkling harbor, and lush green hills.

## Create embeddings

In [53]:
from langchain.embeddings import CohereEmbeddings
co = CohereEmbeddings(cohere_api_key=key)
# text = "This is a test document."
# query_result = embeddings.embed_query(text)
# doc_result = embeddings.embed_documents([text])
# print(doc_result)


## read source file

In [54]:
def read_urls(file_path):
    '''Function to read URLs from a file'''
    with open(file_path, 'r') as file:
        urls = [url.strip() for url in file.readlines()]
    return urls

urls = read_urls("./source_url.txt")

## load the source

In [55]:
from langchain.document_loaders import WebBaseLoader

# Specify the URL of the website you want to load
all_documents = []

# Load the website content for each URL and add it to the list
for url in urls:
    loader = WebBaseLoader(url)
    documents = loader.load()
    all_documents.extend(documents)

## split the website content into smaller chunks

In [56]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import VectorDBQA
from langchain.llms import Cohere
import os

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_documents)

os.environ["COHERE_API_KEY"] = key

In [59]:
embeddings = CohereEmbeddings()
db = Chroma.from_documents(docs, embeddings)

## QnA testing

In [60]:
qa = VectorDBQA.from_chain_type(llm=Cohere(), chain_type="stuff", vectorstore=db)

/Users/apple/Desktop/rag-chatbot/.venv/lib/python3.11/site-packages/langchain/chains/retrieval_qa/base.py:256: UserWarning: `VectorDBQA` is deprecated - please use `from langchain.chains import RetrievalQA`
  warnings.warn(


In [61]:
query = "How do I update expert advisors or indicators from the mql5.com market to latest versions?"
qa.invoke(query)

{'query': 'How do I update expert advisors or indicators from the mql5.com market to latest versions?',
 'result': ' To update your expert advisors or indicators from the MQL5.com market to the latest versions, you can follow these steps:\n\n1. Log in to your MQL5.com account and go to the "Market" section.\n2. Find and select the item that you want to update.\n3. If an update is available, there will be an "Update" button shown on the right side of the page. Click on that button.\n4. A pop-up window will appear. Click on the "Download update" button to start the update process.\n5. Once the update is downloaded, you can copy it to your MT4 or MT5 platform\'s Experts or Indicators folder to replace the existing file. \n\nNote: Make sure you have the appropriate currency on your MQL5.com account to pay for any paid resources, if necessary. \n\nKeeping your EA\'s and indicators up to date can help you stay current with any new features and bug fixes from the developer. '}

In [62]:
invalid_query = "where can i buy icecream?"
qa.invoke(invalid_query)

{'query': 'where can i buy icecream?',
 'result': " I don't know the answer to this question as it is not mentioned in the context nor is it a question related to the previous context. \nI can provide ice cream options if you like however I need to know the location you are looking to purchase ice cream. "}